In [1]:
import os
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup, Comment

# Constants
CBB_BASE_URL = "https://www.sports-reference.com/cbb/players"
PLAYER_IDS_CSV = "player_ids.csv"  # Your existing NBA player IDs CSV with PlayerName column
COLLEGE_STATS_CSV = "college_players_stats.csv"
START_YEAR = 2010  # Filter seasons from this year onward (2010-11 season)

def get_driver():
    options = Options()
    options.binary_location = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('window-size=1920,1080')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def normalize_name_for_college_id(player_name):
    # Lowercase, replace spaces with hyphens, keep existing hyphens, append -1
    name = player_name.lower()
    name = re.sub(r'\s+', '-', name)
    college_id = f"{name}-1"
    return college_id

from io import StringIO

def extract_table_from_html(soup, table_id):
    # Try to find table directly
    table = soup.find('table', id=table_id)
    if table:
        return pd.read_html(StringIO(str(table)))[0]
    # If not found, try inside HTML comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment_soup = BeautifulSoup(comment, 'html.parser')
        table = comment_soup.find('table', id=table_id)
        if table:
            return pd.read_html(StringIO(str(table)))[0]
    return None

def filter_stats_by_season(df, start_year=START_YEAR):
    if df is None or df.empty:
        return None
    if 'Season' not in df.columns:
        return None
    def season_year(season_str):
        try:
            return int(season_str[:4])
        except:
            return None
    df['SeasonYear'] = df['Season'].apply(season_year)
    filtered_df = df[df['SeasonYear'] >= start_year]
    if filtered_df.empty:
        return None
    filtered_df = filtered_df.drop(columns=['SeasonYear'])
    return filtered_df

def scrape_college_stats(driver, player_name):
    college_id = normalize_name_for_college_id(player_name)
    url = f"{CBB_BASE_URL}/{college_id}.html"
    driver.get(url)
    time.sleep(3)  # polite delay to allow page load

    # Check if page exists
    if "Page Not Found" in driver.page_source or "404" in driver.title:
        print(f"College page not found for {player_name} ({college_id})")
        return None, None

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    per_game_df = extract_table_from_html(soup, 'players_per_game')
    advanced_df = extract_table_from_html(soup, 'players_advanced')

    # Filter by season year
    per_game_df = filter_stats_by_season(per_game_df)
    advanced_df = filter_stats_by_season(advanced_df)

    return per_game_df, advanced_df

def append_college_stats_to_csv(player_name, college_id, per_game_df, advanced_df, output_csv=COLLEGE_STATS_CSV):
    if per_game_df is None and advanced_df is None:
        print(f"Skipping {player_name} ({college_id}) because no college stats from {START_YEAR} onwards.")
        return

    # Check if merge keys exist in both dataframes before merging
    merge_keys = ['Season', 'Team']

    # Verify columns exist
    per_game_has_keys = per_game_df is not None and all(key in per_game_df.columns for key in merge_keys)
    advanced_has_keys = advanced_df is not None and all(key in advanced_df.columns for key in merge_keys)

    if per_game_df is not None:
        per_game_df = per_game_df.copy()
        per_game_df['PlayerName'] = player_name
        per_game_df['CollegeID'] = college_id
        print(f"Per-game columns for {player_name}: {per_game_df.columns.tolist()}")

    if advanced_df is not None:
        advanced_df = advanced_df.copy()
        advanced_df['PlayerName'] = player_name
        advanced_df['CollegeID'] = college_id
        print(f"Advanced columns for {player_name}: {advanced_df.columns.tolist()}")

    if per_game_has_keys and advanced_has_keys:
        merged_df = pd.merge(
            per_game_df,
            advanced_df,
            on=merge_keys,
            suffixes=('_per_game', '_adv'),
            how='outer'
        )
        merged_df['PlayerName'] = player_name
        merged_df['CollegeID'] = college_id
    elif per_game_df is not None and not per_game_has_keys:
        print(f"Skipping {player_name} ({college_id}) because per_game_df missing required columns {merge_keys}.")
        return
    elif advanced_df is not None and not advanced_has_keys:
        print(f"Skipping {player_name} ({college_id}) because advanced_df missing required columns {merge_keys}.")
        return
    elif per_game_df is not None:
        merged_df = per_game_df
    else:
        merged_df = advanced_df

    if not os.path.isfile(output_csv):
        merged_df.to_csv(output_csv, index=False)
    else:
        merged_df.to_csv(output_csv, mode='a', header=False, index=False)
    print(f"Appended college stats for {player_name} ({college_id})")

def main():
    driver = get_driver()
    if not os.path.isfile(PLAYER_IDS_CSV):
        print(f"Player IDs CSV '{PLAYER_IDS_CSV}' not found. Please provide it with a 'PlayerName' column.")
        return

    df_players = pd.read_csv(PLAYER_IDS_CSV)
    total_players = len(df_players)

    start_index = 0  # default to start from first player

    if os.path.isfile(COLLEGE_STATS_CSV):
        try:
            df_existing = pd.read_csv(COLLEGE_STATS_CSV)
            if 'CollegeID' in df_existing.columns and not df_existing.empty:
                last_college_id = df_existing['CollegeID'].iloc[-1]
                print(f"Last scraped CollegeID in CSV: {last_college_id}")

                # Find index of last scraped player in df_players
                # Normalize player names to college IDs for matching
                college_ids_list = df_players['PlayerName'].apply(normalize_name_for_college_id).tolist()
                if last_college_id in college_ids_list:
                    last_index = college_ids_list.index(last_college_id)
                    start_index = last_index + 1
                    print(f"Resuming scraping from player index {start_index} ({df_players.iloc[start_index]['PlayerName'] if start_index < total_players else 'End of list'})")
                else:
                    print("Last scraped CollegeID not found in player list, starting from beginning.")
            else:
                print("Existing CSV is empty or missing 'CollegeID' column, starting from beginning.")
        except Exception as e:
            print(f"Warning: Could not read existing stats CSV: {e}")

    for idx in range(start_index, total_players):
        player_name = df_players.iloc[idx]['PlayerName']
        college_id = normalize_name_for_college_id(player_name)

        print(f"Scraping college stats for {player_name} ({college_id}) [{idx+1}/{total_players}]")
        per_game_df, advanced_df = scrape_college_stats(driver, player_name)
        if per_game_df is None and advanced_df is None:
            print(f"No college stats found for {player_name}, skipping.")
            continue
        append_college_stats_to_csv(player_name, college_id, per_game_df, advanced_df)
        time.sleep(3)  # polite delay between requests

    driver.quit()

if __name__ == "__main__":
    main()

Last scraped CollegeID in CSV: antonio-blakeney-1
Resuming scraping from player index 395 (Will Blalock)
Scraping college stats for Will Blalock (will-blalock-1) [396/5313]
No college stats found for Will Blalock, skipping.
Scraping college stats for George Blaney (george-blaney-1) [397/5313]
No college stats found for George Blaney, skipping.
Scraping college stats for Lance Blanks (lance-blanks-1) [398/5313]
No college stats found for Lance Blanks, skipping.
Scraping college stats for Ricky Blanton (ricky-blanton-1) [399/5313]
No college stats found for Ricky Blanton, skipping.
Scraping college stats for Andray Blatche (andray-blatche-1) [400/5313]
College page not found for Andray Blatche (andray-blatche-1)
No college stats found for Andray Blatche, skipping.
Scraping college stats for Mookie Blaylock (mookie-blaylock-1) [401/5313]
No college stats found for Mookie Blaylock, skipping.
Scraping college stats for Eric Bledsoe (eric-bledsoe-1) [402/5313]
No college stats found for Eric